In [1]:
import pandas as pd 
import numpy as np
import sys
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
df = pd.read_csv('data/winequality-red.csv', sep = ';')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [5]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [6]:
has_na = df.isna().any().any()
has_na

False

In [7]:
X = df.drop(columns = ['quality'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 522)


In [8]:
models1 = {
    "dummy": DummyClassifier(random_state=522), 
    "KNN": KNeighborsClassifier(),
    "RBF SVM": SVC(random_state=123), 
    'Ridge model':Ridge(),
    'linear SVC':SVC(kernel = 'linear'),
    'decision tree': DecisionTreeClassifier(),
}

In [9]:
from sklearn.compose import make_column_transformer


results1 = []

for name, model in models1.items():
    pipeline =  make_pipeline(StandardScaler(), model) 
    scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, n_jobs=-1)
 #   mean_std_cross_val_scores
    results1.append({
        'model': name,
        'test_score': np.mean(scores['test_score']),
        'train_score': np.mean(scores['train_score']),
        'fit_time': np.mean(scores['fit_time']),
        'score_time': np.mean(scores['score_time'])
    })

results_df1 = pd.DataFrame(results1)
results_df1.set_index('model', inplace=True)
results_df1

,test_score,train_score,fit_time,score_time
model,,,,
dummy,0.440575,0.440572,0.003000,0.001000
KNN,0.551381,0.699952,0.003400,0.005999
RBF SVM,0.605898,0.676940,0.021599,0.008600
Ridge model,0.332069,0.356684,0.008600,0.001200
linear SVC,0.570179,0.596963,0.024200,0.003400
decision tree,0.591576,1.000000,0.005399,0.001000


In [10]:
pipe_ridge = make_pipeline(StandardScaler(), Ridge())
pipe_ridge.fit(X_train, y_train)

coeffs = pipe_ridge.named_steps["ridge"].coef_

# Use df.columns to get the feature names if X_train was derived from df
coeff_df = pd.DataFrame(data=coeffs, index=X_train.columns, columns=["Coefficients"])
sorted_coeff_df = coeff_df.sort_values(by="Coefficients", ascending=False)

sorted_coeff_df

C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprec

,Coefficients
alcohol,0.315652
sulphates,0.153658
free sulfur dioxide,0.052899
fixed acidity,0.045094
residual sugar,0.034900
density,-0.019654
citric acid,-0.046625
chlorides,-0.057411
pH,-0.063352
total sulfur dioxide,-0.117559


In [11]:
#drop variables with small coefficients(< 0.05) (free sulfur dioxide, residual sugar, density, citric acid)
X = df.drop(columns = ['quality','free sulfur dioxide', 'residual sugar', 'density', 'citric acid'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 522)
results1 = []

for name, model in models1.items():
    pipeline =  make_pipeline(StandardScaler(), model) 
    scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, n_jobs=-1)
 #   mean_std_cross_val_scores
    results1.append({
        'model': name,
        'test_score': np.mean(scores['test_score']),
        'train_score': np.mean(scores['train_score']),
        'fit_time': np.mean(scores['fit_time']),
        'score_time': np.mean(scores['score_time'])
    })

results_df1 = pd.DataFrame(results1)
results_df1.set_index('model', inplace=True)
results_df1

,test_score,train_score,fit_time,score_time
model,,,,
dummy,0.440575,0.440572,0.002400,0.001000
KNN,0.560314,0.703306,0.004000,0.004999
RBF SVM,0.588057,0.652142,0.019799,0.008200
Ridge model,0.332776,0.350687,0.007400,0.001000
linear SVC,0.555898,0.574396,0.018798,0.003200
decision tree,0.586231,1.000000,0.004199,0.001000


In [12]:
#ridge coefficient
pipe_ridge = make_pipeline(StandardScaler(), Ridge())
pipe_ridge.fit(X_train, y_train)

coeffs = pipe_ridge.named_steps["ridge"].coef_

# Use df.columns to get the feature names if X_train was derived from df
coeff_df = pd.DataFrame(data=coeffs, index=X_train.columns, columns=["Coefficients"])
sorted_coeff_df = coeff_df.sort_values(by="Coefficients", ascending=False)

sorted_coeff_df

C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\User\miniconda3\envs\dsci522_group20_env\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprec

,Coefficients
alcohol,0.323220
sulphates,0.148581
fixed acidity,0.004924
pH,-0.064149
chlorides,-0.064908
total sulfur dioxide,-0.081622
volatile acidity,-0.187494


# ANALYSIS:

In this study, we employed various machine learning models to predict the quality of wine based on its chemical properties. The models used included a Dummy model, K-Nearest Neighbors (KNN), Support Vector Machine (SVM) with Radial Basis Function (RBF) kernel, Ridge Regression, and Linear Support Vector Classification (SVC). These models were rigorously cross-validated with 5 folds to assess their performance. The Python programming language, along with essential packages such as Pandas,scikit-learn was instrumental in conducting this analysis.

The initial performance of each model, as measured by test scores, was as follows:

Dummy Model: 0.437059
KNN: 0.562917
RBF SVM: 0.615313
Ridge Model: 0.340925
Linear SVC: 0.566094

To improve model performance and streamline the feature set, we conducted a coefficient analysis. This analysis led to the exclusion of variables with coefficients less than 0.05, including variables within 'free sulfur dioxide', 'residual sugar', 'density', and 'citric acid'. The updated model performances were:

Dummy Model: 0.437059 (unchanged)
KNN: 0.571541
RBF SVM: 0.606740
Ridge Model: 0.344543
Linear SVC: 0.576244

The new coefficients for the remaining variables were:

Alcohol: 0.322230
Sulphates: 0.145794
Fixed Acidity: 0.011367
pH: -0.059857
Chlorides: -0.067214
Total Sulfur Dioxide: -0.089573
Volatile Acidity: -0.179928

This refined analysis suggests a more focused model, with the reduced feature set enhancing the predictive accuracy of certain models, notably the KNN and Linearthis analysisCincludesa utilizechemicalpass various physicochemical properties of wine, such as acidity, sulfur dioxide levels, Based on the results, we will focus on decision tree model, RBF SVM, Ridge for hyperparameter optimizations. ed to influence its quality.
